In [1]:
import astropy.units as u
from astropy.time import Time
from astropy.coordinates import SkyCoord
import numpy as np
from pathlib import Path
from gammapy.data import DataStore, EventList, Observation, Observations
from gammapy.maps import WcsGeom, MapAxis
from gammapy.datasets import MapDataset
from gammapy.makers import MapDatasetMaker, FoVBackgroundMaker, SafeMaskMaker

Gammapy support for parallelisation with ray is still a prototype and is not fully functional.


In [2]:
DL3_direc = "/Volumes/WorkData/hess/fits/hap_fr/hap-16-03_fitscl7/Prod23_Calib0834/ash_stereo_Prod17_Calib0834_thsq64"

In [3]:
data_store = DataStore.from_dir(DL3_direc)

In [4]:
target_pos = SkyCoord(ra=266.816121, dec=-29.966925, unit='deg', frame='icrs')

In [5]:
selection = dict(
    type="sky_circle",
    frame="icrs",
    lon=target_pos.ra,
    lat=target_pos.dec,
    radius="4 deg",
)
selected_obs_table = data_store.obs_table.select_observations(selection)

In [6]:
obs_id = selected_obs_table['OBS_ID']
obs_id_valid = obs_id[np.where(obs_id >= 50563)]

In [7]:
observations = data_store.get_observations(obs_id_valid, required_irf="full-enclosure")

In [8]:
obs = observations[0]

In [9]:
energy = MapAxis.from_energy_bounds(0.1, 10, 5, per_decade=True, unit='TeV')
print(energy)

MapAxis

	name       : energy    
	unit       : 'TeV'     
	nbins      : 10        
	node type  : edges     
	edges min  : 1.0e-01 TeV
	edges max  : 1.0e+01 TeV
	interp     : log       



In [10]:
energy_true = MapAxis.from_energy_bounds(0.05, 50, 8, per_decade=True, unit='TeV', name='energy_true')

In [11]:
geom = WcsGeom.create(skydir=target_pos, width=4 * u.deg, binsz=0.02 * u.deg, frame='icrs', axes=[energy])

In [12]:
stacked = MapDataset.create(
    geom=geom, energy_axis_true=energy_true, name="stacked"
)

In [13]:
offset_max = 2.5 * u.deg
maker = MapDatasetMaker()
maker_safe_mask = SafeMaskMaker(
    methods=["offset-max", "aeff-max"], offset_max=offset_max
)
maker_fov = FoVBackgroundMaker()

In [14]:
dataset = maker.run(stacked, obs)
dataset = maker_safe_mask.run(dataset, obs)
dataset = maker_fov.run(dataset)

First try: _gti
First except: _gti
Second try: _gti
First try: aeff
First except: aeff
Second try: aeff
First try: _pointing
First except: _pointing
Second try: _pointing
Second except: _pointing
HDU 'EVENTS' not found
First try: _events
First except: _events
Second try: _events
First try: _events
First except: _events
Second try: _events
First try: _pointing
First try: counts
First try: counts
First try: _events
First except: _events
Second try: _events
First try: exposure
First try: aeff
First except: aeff
Second try: aeff
First try: _gti
First except: _gti
Second try: _gti
First try: _gti
First except: _gti
Second try: _gti
First try: _gti
First except: _gti
Second try: _gti
First try: _pointing
First try: _pointing
First try: _pointing
First try: _gti
First except: _gti
Second try: _gti
First try: aeff
First except: aeff
Second try: aeff
First try: counts
First try: bkg
First except: bkg
Second try: bkg
First try: _pointing
First try: _pointing
First try: _gti
First except: _gti
Se